In [14]:
import cvxpy as cp
import openpyxl
import numpy as np

# Load data from Excel
def load_data(file_path):
    wb = openpyxl.load_workbook(r"C:\Users\bwvir\Downloads\RadioData1.xlsx")
    sheet = wb.active
    data = []

    for row in sheet.iter_rows(values_only=True):
        data.append(row)

    
    headers = data[0]    #first row of input data is the headers
    dj_data = data[1:]    #rest is data
    return headers, dj_data

# Process availability data into a matrix
def process_availability(dj_data):
    dj_names = [dj[1] for dj in dj_data]   #DJ name is stored in the second collumn (dj[1]) 
    availability = {}
    priority_scores = []
    
    for dj in dj_data:
        dj_availability = []
        for i, day_slots in enumerate(dj[3:10]):  # Monday to Sunday slots
            if day_slots:
                slots = day_slots.split(", ")    #turn availability from string to array
                dj_availability.extend((i, slot) for slot in slots)
        availability[dj[1]] = dj_availability   #connect DJ name to their availability
        priority_score = (dj[15])    #priority score is 16th column of data (dj[15])
        priority_scores.append(priority_score)
    return dj_names, availability, np.array(priority_scores[0:82])

# Define the optimization problem
def optimize_schedule(dj_names, availability, priority_scores):
    num_djs = len(dj_names)
    num_slots = 70  # Total slots in the week

    # Binary decision variables
    x = cp.Variable((num_djs, num_slots), boolean=True)

    # Constraints
    constraints = []
    for dj_index, slots in enumerate(availability.values()):
        dj_av_index = []
        for day, slot in slots:
            # Add a constraint for DJ availability
            # Extract the hour and period (AM/PM)
            hour, period = slot.split()[0], slot.split()[1]

            # Convert to 24-hour format
            hour = int(hour)
            if period == "PM" and hour != 12:
                hour += 12
            if period == "AM" and hour == 12:
                hour = 0

            # Calculate the index
            slot_index = day * 10 + hour - 9  # Convert time to index
            dj_av_index.append(slot_index)
        
        #If a DJ is not available during a slot, they cannot have that slot
        for i in range(0,69):
            if i not in dj_av_index:
                constraints.append(x[dj_index, i] == 0)
    
       
    # Ensure every slot is filled by exactly one DJ
    constraints.append(cp.sum(x, axis=0) == 1)

    # Ensure each DJ is assigned at most one slot
    constraints.append(cp.sum(x, axis=1) <= 1)

        
    # Objective function: Maximize total priority score
    objective = cp.Maximize(cp.sum(cp.multiply(priority_scores, cp.sum(x, axis=1))))
    
    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve()

    return x.value

# Save the schedule to Excel
def save_schedule(file_path, dj_names, schedule, priority_scores):
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.title = "Schedule"
    sheet.append(["DJ Name", "Slot Index", "Priority Score"])    #append headers
    # Write schedule
    for dj_index, assignments in enumerate(schedule):
        for slot_index, assignment in enumerate(assignments):
            if assignment > 0.5:  # Binary decision
                sheet.append([dj_names[dj_index], slot_index, priority_scores[dj_index]])

    wb.save(file_path)

# Main workflow
output_file = r"C:\Users\bwvir\Downloads\optimized_schedule.xlsx"  # Output Excel file

headers, dj_data = load_data(r"C:\Users\bwvir\Downloads\RadioData1.xlsx")
dj_names, availability, priority_scores = process_availability(dj_data)
schedule = optimize_schedule(dj_names, availability, priority_scores)
save_schedule(output_file, dj_names, schedule, priority_scores)